In [1]:
from __future__ import absolute_import

from datetime import datetime
import json
import os
import shutil
from timeit import default_timer as timer

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
import numpy as np
import keras.backend as K
from keras.utils import to_categorical
from cleverhans.utils_keras import KerasModelWrapper
# from cleverhans.attacks_tf import pgd_attack
from cleverhans.attacks import MadryEtAl

from lib.utils import load_gtsrb
from stn.conv_model import conv_model_no_color_adjust, build_cnn_no_stn

with open('config.json') as config_file:
    config = json.load(config_file)

# Setting up training parameters
tf.set_random_seed(config['random_seed'])

max_num_training_steps = config['max_num_training_steps']
num_output_steps = config['num_output_steps']
num_summary_steps = config['num_summary_steps']
num_checkpoint_steps = config['num_checkpoint_steps']

batch_size = config['training_batch_size']

# Load data
X_train, y_train, X_val, y_val, X_test, y_test = load_gtsrb()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

# Obtain Image Parameters
img_rows, img_cols, nchannels = X_train.shape[1:4]
nb_classes = y_train.shape[1]

sess = tf.Session()
K.set_session(sess)

# Set u
global_step = tf.train.get_or_create_global_step()
model = build_cnn_no_stn()

Using TensorFlow backend.


In [2]:
wrap_model = KerasModelWrapper(model)
x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols, nchannels))
y = tf.placeholder(tf.float32, shape=(None, nb_classes))
y = tf.stop_gradient(y)

In [3]:
# Set up Cleverhans adversarial examples
pgd_params = {'eps': config['eps'],
              'eps_iter': config['eps_iter'],
              'nb_iter': config['nb_iter'],
              'clip_min': 0.,
              'clip_max': 1.,
              'rand_init': True}
pgd = MadryEtAl(wrap_model, sess=sess)
x_adv = pgd.generate(x, **pgd_params)

# Get cross entropy loss of perturbed input
# y_adv = model(x_adv)
logits_adv = wrap_model.get_logits(x_adv)
train_step = tf.nn.softmax_cross_entropy_with_logits_v2(
    labels=y, logits=logits_adv)

In [4]:
sess.run(tf.global_variables_initializer())
training_time = 0.0

# Main training loop
for ii in range(max_num_training_steps):
    
    # Get batch of training samples
    # x_batch, y_batch = mnist.train.next_batch(batch_size)
    ind = np.random.randint(len(X_train), size=batch_size)
    x_batch, y_batch = X_train[ind], y_train[ind]

    # Compute Adversarial Perturbations
#     start = timer()
#     x_batch_adv = attack.perturb(x_batch, y_batch, sess)
#     end = timer()
#     training_time += end - start

#     nat_dict = {model.x_input: x_batch,
#                 model.y_input: y_batch}

#     adv_dict = {model.x_input: x_batch_adv,
#                 model.y_input: y_batch}

    # Output to stdout
#     if ii % num_output_steps == 0:
#         nat_acc = sess.run(model.accuracy, feed_dict=nat_dict)
#         adv_acc = sess.run(model.accuracy, feed_dict=adv_dict)
#         print('Step {}:    ({})'.format(ii, datetime.now()))
#         print('    training nat accuracy {:.4}%'.format(nat_acc * 100))
#         print('    training adv accuracy {:.4}%'.format(adv_acc * 100))
#         if ii != 0:
#             print('    {} examples per second'.format(
#                 num_output_steps * batch_size / training_time))
#             training_time = 0.0
    # Tensorboard summaries
#     if ii % num_summary_steps == 0:
#         summary = sess.run(merged_summaries, feed_dict=adv_dict)
#         summary_writer.add_summary(summary, global_step.eval(sess))

    # Write a checkpoint
#     if ii % num_checkpoint_steps == 0:
#         saver.save(sess,
#                    os.path.join(model_dir, 'checkpoint'),
#                    global_step=global_step)

    # Actual training step
    feed_dict = {x: x_batch, y: y_batch}
    start = timer()
    sess.run(train_step, feed_dict=feed_dict)
    end = timer()
    training_time += end - start

InvalidArgumentError: Invalid loop structure: Mismatched parent frames for "while/while_context": "while/while_context" vs "". This is an internal bug, please file a bug report with instructions on how to reproduce the error.

In [5]:
x_batch.shape

(50, 32, 32, 3)

In [3]:
# Set up Cleverhans adversarial examples
pgd_params = {'eps': config['eps'],
              'eps_iter': config['eps_iter'],
              'nb_iter': config['nb_iter'],
              'clip_min': 0.,
              'clip_max': 1.,
              'rand_init': True}
pgd = MadryEtAl(wrap_model, sess=sess)

In [4]:
train_step = tf.nn.softmax_cross_entropy_with_logits_v2(
    labels=y, logits=wrap_model.get_logits(x))

sess.run(tf.global_variables_initializer())
training_time = 0.0

# Main training loop
for ii in range(max_num_training_steps):
    print(ii)
    
    # Get batch of training samples
    # x_batch, y_batch = mnist.train.next_batch(batch_size)
    ind = np.random.randint(len(X_train), size=batch_size)
    x_batch, y_batch = X_train[ind], y_train[ind]

    # Compute Adversarial Perturbations
    start = timer()
    x_batch_adv = pgd.generate_np(x_batch, **pgd_params)
    end = timer()
    training_time += end - start

#     nat_dict = {model.x_input: x_batch,
#                 model.y_input: y_batch}

#     adv_dict = {model.x_input: x_batch_adv,
#                 model.y_input: y_batch}

    # Output to stdout
#     if ii % num_output_steps == 0:
#         nat_acc = sess.run(model.accuracy, feed_dict=nat_dict)
#         adv_acc = sess.run(model.accuracy, feed_dict=adv_dict)
#         print('Step {}:    ({})'.format(ii, datetime.now()))
#         print('    training nat accuracy {:.4}%'.format(nat_acc * 100))
#         print('    training adv accuracy {:.4}%'.format(adv_acc * 100))
#         if ii != 0:
#             print('    {} examples per second'.format(
#                 num_output_steps * batch_size / training_time))
#             training_time = 0.0
    # Tensorboard summaries
#     if ii % num_summary_steps == 0:
#         summary = sess.run(merged_summaries, feed_dict=adv_dict)
#         summary_writer.add_summary(summary, global_step.eval(sess))

    # Write a checkpoint
#     if ii % num_checkpoint_steps == 0:
#         saver.save(sess,
#                    os.path.join(model_dir, 'checkpoint'),
#                    global_step=global_step)

    # Actual training step
    feed_dict = {x: x_batch_adv, y: y_batch}
    start = timer()
    sess.run(train_step, feed_dict=feed_dict)
    end = timer()
    training_time += end - start

0


InvalidArgumentError: Invalid loop structure: Mismatched parent frames for "while/while_context": "while/while_context" vs "". This is an internal bug, please file a bug report with instructions on how to reproduce the error.

In [6]:
model.layers[-1].output

<tf.Tensor 'dense_1/Softmax:0' shape=(?, 43) dtype=float32>

In [1]:
import json
import logging
import os
import time
from keras import backend as K

from cleverhans.attacks import CarliniWagnerL2, FastGradientMethod, MadryEtAl
from cleverhans.utils import AccuracyReport, set_log_level
from cleverhans.utils_keras import KerasModelWrapper
from cleverhans.utils_tf import model_eval
from lib.utils import load_gtsrb
from parameters import *
from small_net import *
# Load and set up all models
from stn.conv_model import build_cnn_no_stn, conv_model_no_color_adjust

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Using TensorFlow backend.


In [4]:
from adv_model import CnnGtsrb
model = CnnGtsrb('model1', 43)

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


In [12]:
w = model.get_params()
sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
w_ = sess.run(w)
print(w_)

[array([[[[-4.79085632e-02,  1.00141101e-01, -8.94975588e-02, ...,
          -7.23946020e-02, -4.31743190e-02,  9.58527848e-02],
         [-2.57828999e-02, -8.20656866e-02,  2.85947174e-02, ...,
          -8.26623663e-03, -5.28259017e-02,  9.76454914e-02],
         [-8.67988169e-02,  2.80705672e-02,  3.74383316e-03, ...,
          -5.43374307e-02,  3.32394280e-02, -1.54762752e-02]],

        [[ 9.02374461e-02,  4.53412645e-02,  4.79820035e-02, ...,
          -4.94347624e-02,  9.12818760e-02, -1.14027662e-02],
         [-1.23974420e-02, -5.19350730e-02, -1.46865696e-01, ...,
           3.54789272e-02,  6.33169264e-02, -1.19996637e-01],
         [-1.69283412e-02,  5.45913689e-02, -1.12335615e-01, ...,
          -7.25039989e-02, -2.54746526e-02, -1.14505048e-04]],

        [[ 2.12005060e-02, -2.18180045e-02,  4.86336928e-03, ...,
          -5.19904383e-02, -1.00145921e-01, -5.81413731e-02],
         [ 1.30442187e-01,  6.47103488e-02,  2.12536938e-02, ...,
          -1.05862185e-01, -9.820